## **Ajuste Fino de um Modelo de IA Generativa para Tarefas de Pergunta e Respostas.**

Neste notebook, irei trabalhar com dados extraídos do site oficial da Secretaria de Estado de Desenvolvimento Social de Goiás, com o objetivo de ajustar um modelo pré-treinado para lidar com perguntas e respostas relacionadas aos programas e benefícios oferecidos pela secretária. Para se adequar com a nossa necessidade, o modelo passará por um processo de ajuste fino. Primeiramente, iremos realizar um ajuste fino (Fine Tune, em inglês) total, onde todas as camadas do modelo pré-treinado serão modificadas, e depois apresentaremos o método PEFT (Ajuste Fino Eficiente de Parâmetros) que gasta menos recursos computacionais, e modifica apenas algumas camadas do modelo, conseguindo assim que ele retenha o conhecimento que já tem e evite o esquecimento catastrófico. Ao final, avaliaremos a precisão dos  dois modelos ajustados e determinaremos sua adequação para as tarefas definidas.

#### Baixando bibliotecas necessárias:

- transformers[torch] : A bibilioteca Transformers pode ser entendida como uma ferramenta poderosa para trabalhar com Processamento de Linguagem Natural. Aqui instalamos o Transformers e a biblioteca de deep learning respectiva apenas numa linha.
- Datasets :  A biblioteca Datasets, também desenvolvida pela Hugging Face, é uma coleção de conjuntos de dados prontos para uso em uma variedade de tarefas de processamento de linguagem natural (PLN) e aprendizado de máquina (ML).
- Evaluate: Com essa biblioteca você ganha acesso a dezenas de métodos de avaliação para diferentes modelos, o que fdacilita a acilita a avaliação de modelos de machine learning e datasets.
- accelerate : A biblioteca accelerate da Hugging Face é uma ferramenta poderosa criada para facilitar o treinamento distribuído de modelos de machine learning, especialmente modelos baseados em Transformers.

In [7]:
#baixando arquivos necessários

!pip install transformers[torch] datasets evaluate pandas
!pip install accelerate -U

 Mesmo que as bibliotecas já estejam instaladas em seu ambiente Python, vocêdevea importá-las em seu script para poder acessar suas funcionalidades. O comando import é essencial para isso.

In [8]:
#importando partes específicas das bibliotecas que serão importantes para o fine-tune
from transformers import AutoTokenizer, pipeline, AutoModelForPreTraining, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset

#### Escolhendo o modelo

Quando vamos realizar um ajuste fino, sempre é interessante escolhermos modelos que foram pré-treinados para tarefas semelhantes. 

Agora podemos aproveitar para configurar o tokenizador que foi utilizado no treinamento do modelo escolhido. Graças ao **AutoTokenizer** que importamos no inicio do codigo, não precisamos nos preocupar em buscar qual forma de tokenização foi utilizada. Basta utilizar o AutoTokenizer, pois ele faz isso automaticamente.


In [9]:
#Escolhendo o modelo para realizar o Ajuste Fino
model_name = "bigscience/bloom-560m" 
model = AutoModelForPreTraining.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

#### Preparando os dados

Quando vamos realizar um Fine-Tune, antes de qualquer coisa, precisamos preparar os dados que serão utilizados para realizar o ajuste do modelo.

Os dados utilizados neste algoritmo foram criados com base nos beneficios oferecidos pela Secretaria de Estado de Desenvolvimento Social de Goiás. Essas informações podem ser consultadas neste [link]("https://goias.gov.br/social/#").

Para facilitar a utilização e simplificar nosso código, realizamos toda a trativa dos dados em outro notebook. Para acessá-lo, basta clicar neste [link](./Dados/TrabalhandoDados.ipynb).

In [10]:
data_file = {"train": "./Dados/treino.csv", "test":'./Dados/teste.csv'}
dataSetCompleto = load_dataset("csv", data_files=data_file)

#atribuindo a variavel dados de tewste e de treino

print(dataSetCompleto)

treino = dataSetCompleto['train']
teste = dataSetCompleto['test']

DatasetDict({
    train: Dataset({
        features: ['pergunta', 'resposta'],
        num_rows: 2120
    })
    test: Dataset({
        features: ['pergunta', 'resposta'],
        num_rows: 531
    })
})


#### Tokenização

Chegamos no momento em que começamos a transformar nossos dados em dados que o modelo escolhido consiga compreender, ou seja, chegamos na fase da Tokenização dos dados.

Para facilitar a tokenização dos dados de treinamento e teste, criamos uma função chamada Tokenizer. Essa função receberá os dados a serem tokenizados e retornará os dados tokenizados.os.

In [13]:
#Aqui criamos uma função para facilitar a tokenização

def tokenizar(dados):
    return tokenizer(dados["pergunta"], dados["resposta"], truncation=True, padding="max_length", max_length=526)

#Utilizamos a função .map() para aplicar a tokenização em todos os elementos dos datasets
dataset_teste = teste.map(tokenizar)
dataset_treino = treino.map(tokenizar)

# A função DataCollatorForLanguageModeling() 
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

##### Metricas de avaliação



In [25]:
def compute_metrics(eval_prediction):
    loss = eval_prediction.loss
    perplexity = torch.exp(torch.tensor(loss)).item()
    return {"perplexity": perplexity, "eval_loss": loss}

In [26]:
#configurarndo parametros de treino

training_args = TrainingArguments(
    output_dir='/home/seds/projeto_generativo/modeloBloom1',
    overwrite_output_dir = True,
    evaluation_strategy='epoch',
    num_train_epochs=1,
    per_device_eval_batch_size=2,
    per_device_train_batch_size=3
    
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_treino,
    eval_dataset=dataset_teste,
    data_collator=data_collator,
    compute_metrics=compute_metrics     
)

In [ ]:
trainer.train()

In [ ]:
output_model_dir = "/home/seds/projeto_generativo/modeloBloom1"
model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)